# Time Series Forecasting with Multiple Models

This notebook demonstrates how to use multiple models (ARIMA, ETS, Random Forest, XGBoost, SARIMA, and Mixture of Experts) to forecast time series data for multiple SKUs.

In [ ]:

# Import necessary libraries for processing and modeling
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the data from uploaded files
inventory_data = pd.read_csv('Domestic Auto Inventories.csv', index_col='DATE', parse_dates=True)
production_data = pd.read_csv('Domestic Auto Production.csv', index_col='DATE', parse_dates=True)
sales_data = pd.read_csv('Total Vehicle Sales.csv', index_col='DATE', parse_dates=True)

# Combine all data into a single DataFrame
df = pd.concat([inventory_data, production_data, sales_data], axis=1)

# Fill missing values with forward fill first, then backward fill
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

# Function to calculate performance metrics
def calculate_metrics(true_values, predicted_values):
    mae = mean_absolute_error(true_values, predicted_values)
    mape = mean_absolute_percentage_error(true_values, predicted_values)
    wape = np.sum(np.abs(true_values - predicted_values)) / np.sum(np.abs(true_values))
    return mae, mape, wape

# Training functions for various models
def train_arima(train, test):
    model = ARIMA(train, order=(5, 1, 0)).fit()
    forecast = model.forecast(steps=len(test))
    return forecast

def train_ets(train, test):
    model = ExponentialSmoothing(train, trend='add', seasonal=None).fit()
    forecast = model.forecast(steps=len(test))
    return forecast

def train_rf(train, test):
    X = np.arange(len(train)).reshape(-1, 1)
    y = train.values
    model = RandomForestRegressor(n_estimators=100).fit(X, y)
    forecast = model.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
    return forecast

def train_xgb(train, test):
    X = np.arange(len(train)).reshape(-1, 1)
    y = train.values
    model = XGBRegressor(n_estimators=100).fit(X, y)
    forecast = model.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
    return forecast

def train_sarima(train, test):
    model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 0, 12)).fit()
    forecast = model.forecast(steps=len(test))
    return forecast

def train_mixture_of_experts(combined_forecasts, test):
    meta_model = LinearRegression()
    meta_model.fit(combined_forecasts, test)
    meta_forecast = meta_model.predict(combined_forecasts)
    return meta_forecast

# Initialize DataFrame to store results
results = pd.DataFrame(columns=['SKU', 'Model', 'MAE', 'MAPE', 'WAPE'])
moe_results = pd.DataFrame(columns=['SKU', 'Model', 'MAE', 'MAPE', 'WAPE'])

# Loop through each SKU column in the DataFrame
for sku in df.columns:
    print(f"Processing SKU: {sku}")
    sku_data = df[sku].dropna()
    train, test = sku_data[sku_data.index < '2024-01-01'], sku_data[sku_data.index >= '2024-01-01']

    # Train each model and get forecasts
    forecasts = {
        'ARIMA': train_arima(train, test),
        'ETS': train_ets(train, test),
        'Random Forest': train_rf(train, test),
        'XGBoost': train_xgb(train, test),
        'SARIMA': train_sarima(train, test)
    }
    
    for model_name, forecast in forecasts.items():
        mae, mape, wape = calculate_metrics(test, forecast)
        results = results.append({
            'SKU': sku,
            'Model': model_name,
            'MAE': mae,
            'MAPE': mape,
            'WAPE': wape
        }, ignore_index=True)
    
    # Prepare combined forecasts for Mixture of Experts
    combined_forecasts = np.vstack(list(forecasts.values())).T
    meta_forecast = train_mixture_of_experts(combined_forecasts, test)
    mae, mape, wape = calculate_metrics(test, meta_forecast)
    moe_results = moe_results.append({
        'SKU': sku,
        'Model': 'Mixture of Experts',
        'MAE': mae,
        'MAPE': mape,
        'WAPE': wape
    }, ignore_index=True)

# Combine results
all_results = pd.concat([results, moe_results], ignore_index=True)
all_results = all_results.sort_values(by=['SKU', 'MAE', 'MAPE', 'WAPE'], ascending=True)
print(all_results)
